In [ ]:
%matplotlib inline
from trading_gym.env import TradeEnv
import random
import matplotlib
import matplotlib.pylab as plt
matplotlib.use('tkAgg')

def custom_obs_features_func(history, info):
    return history.obs_list

env = TradeEnv(data_path='eurusd_patterns_10_test2_slope_trend_pro.lite1-6.csv',nav=1000, data_kwargs={'use_ta': False})


#env = TradeEnv(data_path='eurusd_patterns_10_test2_slope_trend_pro.lite1-3.csv',ops_shape=[],get_obs_features_func=custom_obs_features_func,nav=1000, data_kwargs={'use_ta': False})

done = False
rewards = 0
obs = env.reset()
for i in range(101):
    action = random.sample([0, 1, 2], 1)[0]
    obs, reward, done, info = env.step(action)
    
    
    #env.render()
    #rewards += reward

    if(i%100 == 0):
        print('{}K ({})'.format(i+1,reward))

        print(obs.shape[0])
      
env.close()
        

In [ ]:
%matplotlib inline
from trading_gym.env import TradeEnv
import random
import matplotlib
import matplotlib.pylab as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gym
matplotlib.use('tkAgg')

class Net(nn.Module):
    def __init__(self, n_states, n_actions, n_hidden):
        super(Net, self).__init__()

        # 輸入層 (state) 到隱藏層，隱藏層到輸出層 (action)
        self.fc1 = nn.Linear(n_states, n_hidden)
        self.fc1.weight.data.normal_(0, 0.1)   # initialization
        self.out = nn.Linear(n_hidden, n_actions)
        self.out.weight.data.normal_(0, 0.1)   # initialization

    def forward(self, x):
        #x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = F.relu(x) # ReLU activation
        actions_value = self.out(x)
        return actions_value
    
class DQN(object):
    def __init__(self, n_states, n_actions, n_hidden, batch_size, lr, epsilon, gamma, target_replace_iter, memory_capacity):
        self.eval_net, self.target_net = Net(n_states, n_actions, n_hidden), Net(n_states, n_actions, n_hidden)

        self.memory = np.zeros((memory_capacity, n_states * 2 + 2)) # 每個 memory 中的 experience 大小為 (state + next state + reward + action)
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=lr)
        self.loss_func = nn.MSELoss()
        self.memory_counter = 0
        self.learn_step_counter = 0 # 讓 target network 知道什麼時候要更新

        self.n_states = n_states
        self.n_actions = n_actions
        self.n_hidden = n_hidden
        self.batch_size = batch_size
        self.lr = lr
        self.epsilon = epsilon
        self.gamma = gamma
        self.target_replace_iter = target_replace_iter
        self.memory_capacity = memory_capacity

    def choose_action(self, state):
        x = torch.unsqueeze(torch.FloatTensor(state), 0)

        # epsilon-greedy
        if np.random.uniform() < self.epsilon: # 隨機
            action = np.random.randint(0, self.n_actions)
        else: # 根據現有 policy 做最好的選擇
            actions_value = self.eval_net.forward(x) # 以現有 eval net 得出各個 action 的分數
            action = torch.max(actions_value, 1)[1].data.numpy()[0] # 挑選最高分的 action

        return action

    def store_transition(self, state, action, reward, next_state):
        # 打包 experience
        transition = np.hstack((state, [action, reward], next_state))

        # 存進 memory；舊 memory 可能會被覆蓋
        index = self.memory_counter % self.memory_capacity
        self.memory[index, :] = transition
        self.memory_counter += 1

    def learn(self):
        # 隨機取樣 batch_size 個 experience
        sample_index = np.random.choice(self.memory_capacity, self.batch_size)
        b_memory = self.memory[sample_index, :]
        b_state = torch.FloatTensor(b_memory[:, :self.n_states])
        b_action = torch.LongTensor(b_memory[:, self.n_states:self.n_states+1].astype(int))
        b_reward = torch.FloatTensor(b_memory[:, self.n_states+1:self.n_states+2])
        b_next_state = torch.FloatTensor(b_memory[:, -self.n_states:])

        # 計算現有 eval net 和 target net 得出 Q value 的落差
        q_eval = self.eval_net(b_state).gather(1, b_action) # 重新計算這些 experience 當下 eval net 所得出的 Q value
        q_next = self.target_net(b_next_state).detach() # detach 才不會訓練到 target net
        q_target = b_reward + self.gamma * q_next.max(1)[0].view(self.batch_size, 1) # 計算這些 experience 當下 target net 所得出的 Q value
        loss = self.loss_func(q_eval, q_target)

        # Backpropagation
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # 每隔一段時間 (target_replace_iter), 更新 target net，即複製 eval net 到 target net
        self.learn_step_counter += 1
        if self.learn_step_counter % self.target_replace_iter == 0:
            self.target_net.load_state_dict(self.eval_net.state_dict())
            

def custom_obs_features_func(history, info):
    return history.obs_list

def custom_reward_func(exchange):
    #plt.plot([exchange.profit])
    #plt.show()
    return exchange.profit

env = TradeEnv(data_path='eurusd_patterns_10_test2_slope_trend_pro.lite1-6.csv',nav=1000, data_kwargs={'use_ta': False})
#env = TradeEnv(data_path='eurusd_patterns_10_test2_slope_trend_pro.lite1-3.csv',ops_shape=[],get_obs_features_func=custom_obs_features_func,nav=1000, data_kwargs={'use_ta': False})

# Environment parameters
n_actions = 3
n_states = 51

# Hyper parameters
n_hidden = 10
batch_size = 32
lr = 0.01                 # learning rate
epsilon = 0.1             # epsilon-greedy
gamma = 0.9               # reward discount factor
target_replace_iter = 100 # target network 更新間隔
memory_capacity = 5000
n_episodes = 1


# 建立 DQN
dqn = DQN(n_states, n_actions, n_hidden, batch_size, lr, epsilon, gamma, target_replace_iter, memory_capacity)

# 學習
for i_episode in range(n_episodes):
    t = 0
    rewards = 0
    obs0 = env.reset()
    while True:
        #env.render()

        # 選擇 action
        action = dqn.choose_action(obs0)
        #obs, reward, done, info = env.step(action)
        obs, reward, done, info = env.step(action)

        # 儲存 experience
        dqn.store_transition(obs0, action, reward, obs)

        # 累積 reward
        rewards += reward

        # 有足夠 experience 後進行訓練
        if dqn.memory_counter > memory_capacity:
            dqn.learn()

        # 進入下一 state
        obs0 = obs

        if done:
            print('Episode finished after {} timesteps, total rewards {}'.format(t+1, rewards))
            
            break

        t += 1

env.close()
        

In [ ]:
%matplotlib inline
from trading_gym.env import TradeEnv
from datetime import datetime
import random
import matplotlib
import matplotlib.pylab as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gym
matplotlib.use('tkAgg')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device =  torch.device("cpu")

def custom_obs_features_func(history, info):
    list = []
    for i in range(len(history.obs_list)):
        list.append(history.obs_list[i].close)
    
    return list

def custom_reward_func(exchange):
    #info
    '''
    {'index': 56, 'date': '2010-01-01 01:04', 'nav': 50000, 'amount': 250000, 'avg_price': 1.4325899999999998,
    'profit': {'total': -282.0124161115024, 'fixed': -272.23990618194, 'floating': -9.7725099295624},
    'buy_at': 52, 'latest_price': 1.43231}
    '''
    #print(exchange.info)
    #print('Profit: {} , floating: {} , fixed: {}'.format(exchange.profit, exchange.floating_profit, exchange.fixed_profit))
    
    # profit , index - 50
    reward = exchange.profit * (exchange.info["index"] - 50) * 0.1
    #print(exchange.info["amount"])
    #print(exchange.available_actions)
    
    return reward

# Hyper Parameters
BATCH_SIZE = 32
LR = 0.01                   # learning rate
EPSILON = 0.9               # greedy policy
GAMMA = 0.9                 # reward discount
TARGET_REPLACE_ITER = 100   # target update frequency
MEMORY_CAPACITY = 5000
env = TradeEnv(data_path='eurusd_patterns_10_test2_slope_trend_pro.lite1-6.csv',
               ops_shape=[],
               get_obs_features_func=custom_obs_features_func,
               get_reward_func=custom_reward_func,
               nav=5000, 
               data_kwargs={'use_ta': False}
              )

#nv = TradeEnv(data_path='eurusd_patterns_10_test2_slope_trend_pro.lite1-6.csv',ops_shape=[],get_obs_features_func=custom_obs_features_func,nav=1000,get_reward_func=custom_reward_func, data_kwargs={'use_ta': False})
#env = TradeEnv(data_path='eurusd_patterns_10_test2_slope_trend_pro.lite1-6.csv',nav=1000, data_kwargs={'use_ta': False})
env = env.unwrapped
N_ACTIONS = 3
N_STATES = 51
ENV_A_SHAPE = 0
n_episodes = 1 #10000
PATH = "./training_game_01.h5"


class Net(nn.Module):
    def __init__(self, ):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(N_STATES, 50).to(device)
        self.fc1.weight.data.normal_(0, 0.1)   # initialization
        self.out = nn.Linear(50, N_ACTIONS).to(device)
        self.out.weight.data.normal_(0, 0.1)   # initialization

    def forward(self, x):
        #print(x)
        #print(x.size())
        x = x.to(device)
        x = self.fc1(x)
        x = F.relu(x)
        #print(x.size())
        actions_value = self.out(x)
        #print(actions_value)
        #print(actions_value.size())
        return actions_value


class DQN(object):
    def __init__(self):
        self.eval_net, self.target_net = Net().to(device), Net().to(device)

        self.learn_step_counter = 0                                     # for target updating
        self.memory_counter = 0                                         # for storing memory
        self.memory = np.zeros((MEMORY_CAPACITY, N_STATES * 2 + 2))     # initialize memory
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)
        self.loss_func = nn.MSELoss()

    def choose_action(self, x):
        #print(x)
        x = torch.unsqueeze(torch.FloatTensor(x), 0).to(device)
        # input only one sample
        if np.random.uniform() < EPSILON:   # greedy
            actions_value = self.eval_net.forward(x)
            action = torch.max(actions_value, 1)[1].data.cpu().numpy()
            action = action[0] if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)  # return the argmax index
            
        else:   # random
            action = np.random.randint(0, N_ACTIONS)
            action = action if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)
            #print("rd")
            #print(action)
        return action

    def store_transition(self, s, a, r, s_):
        transition = np.hstack((s, [a, r], s_))
        # replace the old memory with new memory
        index = self.memory_counter % MEMORY_CAPACITY
        self.memory[index, :] = transition
        self.memory_counter += 1

    def learn(self):
        # target parameter update
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:
            self.target_net.load_state_dict(self.eval_net.state_dict())
        self.learn_step_counter += 1

        # sample batch transitions
        sample_index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE)
        b_memory = self.memory[sample_index, :]
        b_s = torch.FloatTensor(b_memory[:, :N_STATES]).to(device)
        b_a = torch.LongTensor(b_memory[:, N_STATES:N_STATES+1].astype(int)).to(device)
        b_r = torch.FloatTensor(b_memory[:, N_STATES+1:N_STATES+2]).to(device)
        b_s_ = torch.FloatTensor(b_memory[:, -N_STATES:]).to(device)

        # q_eval w.r.t the action in experience
        q_eval = self.eval_net(b_s).gather(1, b_a)  # shape (batch, 1)
        q_next = self.target_net(b_s_).detach()     # detach from graph, don't backpropagate
        q_target = b_r + GAMMA * q_next.max(1)[0].view(BATCH_SIZE, 1)   # shape (batch, 1)
        loss = self.loss_func(q_eval, q_target)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
    

dqn = DQN()


print('\nCollecting experience...')
for i_episode in range(n_episodes):
    t = 0
    rewards = 0
    obs0 = env.reset()
    while True:
        env.render()
        action = dqn.choose_action(obs0)
        observation, reward, done, info = env.step(action)

        #action 0 sell, 1 hold, 2 buy

        # 儲存 experience
        if(info["profit"]["total"] > 50):
            dqn.store_transition(obs0, action, reward, observation)
        
        # 累積 reward
        rewards += reward

        # 有足夠 experience 後進行訓練
        if dqn.memory_counter > MEMORY_CAPACITY:
            dqn.learn()
            #print(dqn.memory_counter)

        # 進入下一 state
        obs0 = observation

        if done:
            torch.save(dqn, PATH)
            print('({}) finished after {} timesteps, total profit: {}, memery: {}'.format(i_episode+1, t+1, info["profit"]["total"],dqn.memory_counter))
            break

        t += 1

env.close()

In [ ]:
%matplotlib inline
from trading_gym.env import TradeEnv
from datetime import datetime
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import random
import matplotlib
import matplotlib.pylab as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gym
matplotlib.use('tkAgg')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device =  torch.device("cpu")

def custom_obs_features_func(history, info):
    list = []
    for i in range(len(history.obs_list)):
        list.append(history.obs_list[i].close)
    
    return list

def custom_reward_func(exchange):
    #info
    '''
    {'index': 56, 'date': '2010-01-01 01:04', 'nav': 50000, 'amount': 250000, 'avg_price': 1.4325899999999998,
    'profit': {'total': -282.0124161115024, 'fixed': -272.23990618194, 'floating': -9.7725099295624},
    'buy_at': 52, 'latest_price': 1.43231}
    '''
    #print(exchange.info)
    #print('Profit: {} , floating: {} , fixed: {}'.format(exchange.profit, exchange.floating_profit, exchange.fixed_profit))
    
    # profit , index - 50
    reward = exchange.profit * (exchange.info["index"] - 50) * 0.001
    #print(exchange.info["amount"])
    #print(exchange.available_actions)
    
    return reward

# Hyper Parameters
BATCH_SIZE = 32
LR = 0.01                   # learning rate
EPSILON = 0.9               # greedy policy
GAMMA = 0.9                 # reward discount
TARGET_REPLACE_ITER = 100   # target update frequency
MEMORY_CAPACITY = 5000
env = TradeEnv(data_path='eurusd_patterns_10_test2_slope_trend_pro.lite1-6.csv',
               ops_shape=[],
               get_obs_features_func=custom_obs_features_func,
               get_reward_func=custom_reward_func,
               nav=5000, 
               data_kwargs={'use_ta': False}
              )

#nv = TradeEnv(data_path='eurusd_patterns_10_test2_slope_trend_pro.lite1-6.csv',ops_shape=[],get_obs_features_func=custom_obs_features_func,nav=1000,get_reward_func=custom_reward_func, data_kwargs={'use_ta': False})
#env = TradeEnv(data_path='eurusd_patterns_10_test2_slope_trend_pro.lite1-6.csv',nav=1000, data_kwargs={'use_ta': False})
env = env.unwrapped
N_ACTIONS = 3
N_STATES = 1
ENV_A_SHAPE = 0
n_episodes = 5000 #10000
PATH = "./training_game_01.h5"

class DQN(object):
    def __init__(self):
        self.eval_net, self.target_net = self._build_model(), self._build_model()

        self.learn_step_counter = 0        # for target updating
        self.memory_counter = 0            # for storing memory
        self.memory = deque(maxlen=2000)   # initialize memory
        self.model = self._build_model()
        
    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=N_STATES, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(N_ACTIONS, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=LR))
        return model

    def choose_action(self, x):
        x = np.array(x)
        #print(x.shape)
        #print(x)
        #x = torch.unsqueeze(torch.FloatTensor(x), 0).to(device)
        # input only one sample
        if np.random.uniform() < EPSILON:   # greedy
            actions_value = self.eval_net.predict(x)
            action = np.argmax(actions_value[0])   
            
        else:   # random
            action = np.random.randint(0, N_ACTIONS)

        return action

    def store_transition(self, s, a, r, s_):
        self.memory.append((s, a, r, s_))

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)
    

dqn = DQN()


print('\nCollecting experience...')
for i_episode in range(n_episodes):
    t = 0
    rewards = 0
    obs0 = env.reset()
    while True:
        #env.render()
        action = dqn.choose_action(obs0)
        observation, reward, done, info = env.step(action)

        #action 0 sell, 1 hold, 2 buy

        # 儲存 experience
        if(info["profit"]["total"] > 0):
            dqn.store_transition(obs0, action, reward, observation)
        
        # 累積 reward
        rewards += reward

        # 有足夠 experience 後進行訓練
        if dqn.memory_counter > MEMORY_CAPACITY:
            dqn.learn()
            #print(dqn.memory_counter)

        # 進入下一 state
        obs0 = observation

        if done:
            torch.save(dqn, PATH)
            print('({}) finished after {} timesteps, total profit: {}, memery: {}'.format(i_episode+1, t+1, info["profit"]["total"],dqn.memory_counter))
            break

        t += 1

env.close()